
# Generating  data

Example on how to generate Gaussian noise under different conditions
(non-stationarity, gaps in the data, multiple detectors)
as well as including monochromatic instrumental artifacts, also known as [spectral lines](https://www.gw-openscience.org/O3/o3speclines/).

0. [The SFT format](#the-sft-format)
0. [Using pyfstat.Writer](#pyfstat.writer)
0. [Inject signal](#inject-signal)
0. [Include realistic effects (optional)](#include_realistic_effects)
  1. [Non-stationary noise](#non-stationarity)
  1. [Gaps](#gaps)
  1. [Narrow instrumental artifacts](#lines)
  1. [Multiple detectors](#multiifo)

First, you install the module `pyfstat`. At this moment, `numpy` will be reinstalled (Google colab preinstalled numpy, but it is not compatible with the pyfstat's requirement). After installing `numpy` and `pyfstat`, you need to restart the notebook. You just follow the popup.

In [ ]:
!pip install pyfstat

You mount your google drive directory on this notebook. After that, you change the working directory to `2024_OpenDataWorkshop_cw`.

In [ ]:
#----------------------
# Setting up the environment
#----------------------
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/2024_OpenDataWorkshop_cw
!ls

In [ ]:
import numpy as np

import pyfstat
import lal
lal.swig_redirect_standard_output_error(True)
from pyfstat.utils import get_sft_as_arrays

# Local module to simplify plotting
import tutorial_utils

logger = pyfstat.set_up_logger(label="1_generating_data", log_level="INFO")

%matplotlib inline

## The SFT format <a name="the-sft-format"></a>

Continuous gravitational-wave signals (CW for short) are quasi-monochromatic long-standing gravitational waves.
Standard models, such as those corresponding to the signal produced by
non-axisymmetric rapidly-spinning neutron stars,
produce narrow-banded signals which are Doppler-modulated due to the motion of the detector in the Solar system.
An accessible and open-access review on the topic can be found [here](https://www.mdpi.com/2218-1997/7/12/474).

Due to the precise localization of a signal in a narrow frequency band,
Fourier transforms are a suitable tool to analyze this sort of data.
These signals, however, are not completely monochromatic,
meaning a full-time Fourier transform could risk smearing a signal across a very broad frequency band.
Instead, we work using *Short Fourier Transforms* (SFTs), which are Fourier transforms of short data segments
(typically around 30 minutes or 1800 seconds).

This data format is specific to [LALSuite](https://lscsoft.docs.ligo.org/lalsuite/),
but PyFstat provides functions (`pyfstat.helper_functions.get_sft_as_arrays`) to read them
as Numpy arrays. The specifications of the SFT data format are freely accessible as a
[technical document](https://dcc.ligo.org/LIGO-T040164/public).

## Using pyfstat.Writer<a name="pyfstat.writer"></a>

The most basic example is to generate Gaussian noise as measured by a single detector.
This operation can be performed using `pyfstat.Writer` as follows:

In [ ]:
# Setup Writer
writer_kwargs = {
    "label": "SingleDetectorGaussianNoise",
    "outdir": "PyFstat_example_data",
    "tstart": 1238166018,  # Starting time of the observation [GPS time]
    "duration": 5 * 86400,  # Duration [seconds]
    "detectors": "H1",  # Detector to simulate, in this case LIGO Hanford
    "F0": 100.0,  # Central frequency of the band to be generated [Hz]
    "Band": 1.0,  # Frequency band-width around F0 [Hz]
    "sqrtSX": 1e-23,  # Single-sided Amplitude Spectral Density of the noise
    "Tsft": 1800,  # Fourier transform time duration
    "SFTWindowType": "tukey",  # Window function to compute short Fourier transforms
    "SFTWindowParam": 0.01,  # Parameter associated to the window function
}
writer = pyfstat.Writer(**writer_kwargs)

# Create SFTs
writer.make_data()

SFT data is saved at the path specified in `writer.sftfilepath `. This binary format can be opened as Numpy arrays using `pyfstat.helper.get_sft_as_arrays` as follows:

In [ ]:
# Read SFT data into numpy arrays and plot real and imaginary parts
frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)
print(type(frequency), type(timestamps), type(fourier_data))

In [ ]:
print(timestamps.keys(), fourier_data.keys())
print(type(timestamps['H1']), type(fourier_data['H1']))

In [ ]:
print(frequency.shape, timestamps['H1'].shape, fourier_data['H1'].shape)

SFT data could contain different detectors which may be operating during different times.
Thus, `timestamps` and `fourier_data` are dictionaries whose keys correspond to detector names (`H1` for LIGO Hanford, `L1` for LIGO Livingston). `timestamps` labels the time at which the data was taken using GPS seconds, while `fourier_data` contains the Fourier amplitude of such data segment. `frequency`, which is common for all the detectors, is a 1D array labeling the frequency bins.

In [ ]:
tutorial_utils.plot_real_imag_spectrograms(
    timestamps["H1"], frequency, fourier_data["H1"]
);

Since we are generating zero-mean white Gaussian noise,
there is a simple relation between the single-sided amplitude spectral density `sqrtSX` $\sqrt{S_{\text{n}}}$
and the variance of the distribution given by
$$
\sigma^2 = \frac{1}{4} T_{\text{SFT}} S_{\text{n}}
$$
where the two $\frac{1}{2}$ factors are due to the use of a *single-sided* ASD
and the fact that this standard deviation applies to both the real and imaginary parts of the Fourier transform.

In [ ]:
theoretical_stdev = np.sqrt(0.25 * writer_kwargs["Tsft"]) * writer_kwargs["sqrtSX"]
tutorial_utils.plot_real_imag_histogram(fourier_data["H1"], theoretical_stdev);

## Inject signal<a name="inject-signal"></a>

We will make SFT data with the signal injected.
The duration of data is 365 days, amplitude spectral density of the noise (`sqrtSX`) is $10^{-23}$. The length of SFT segments is 1800 sec.

We use the following signal parameters:

- $f_\mathrm{gw} = 100\ \mathrm{Hz}$
- $\dot{f} = 1.0 \times 10^{-9}\ \mathrm{Hz/sec}$
- $\ddot{f} = 0\ \mathrm{Hz/sec^2}$
- sky position $(\alpha, \delta) = (0.0, 0.0)$
- amplitude $h_0 = 1.0 \times 10^{-22}$
- $\cos\iota = 1.0$
- $\psi = \Phi_0 = 0.0$

Generation of data will take 3-4 minutes due to long duration.

In [ ]:
writer_kwargs = {
    "label": "SingleDetectorGaussianNoise",
    "outdir": "PyFstat_example_data",
    "tstart": 1238166018,
    "duration": 365 * 86400,
    "detectors": "H1",
    "sqrtSX": 1e-23,
    "Tsft": 1800,
    "SFTWindowType": "tukey",
    "SFTWindowParam": 0.01,
}

signal_parameters = {
    "F0": 100.0,    # GW frequency at initial time [Hz]
    "F1": -1e-9,    # first derivative of frequency (\dot{f}) [Hz/sec]
    "F2": 0.0,      # second derivative of frequency (\ddot{f}) [Hz/sec^2]
    "Alpha": 0.0,   # right ascention of the source [rad]
    "Delta": 0.0,   # declination angle of the source [rad]
    "h0": 1e-22,    # GW amplitude
    "cosi": 1,      # cosine of the inclination angle
    "psi": 0.0,     # polarization angle [rad]
    "phi": 0.0,     # initial phase [rad]
    "tref": writer_kwargs["tstart"],
}

writer = pyfstat.Writer(**writer_kwargs, **signal_parameters)
writer.make_data()
frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

Here, we plot a spectrogram. The signal amplitude is 10 times larger than the noise strength (h_0 = 1e-22, sqrtSX = 1e-23). So, you can see the signal by eye.

Frequency track has several features.
1. You can check the effect of $\dot{f}$ (`F1`) by comparing the frequencies at initial and final SFT. The frequency change is $$\mathrm{duration} \times \dot{f} \simeq (3\times 10^7 \mathrm{sec}) \times (10^{-9} \mathrm{Hz/sec}) = 0.03 \mathrm{Hz}.$$ This change can be observed in the spectrogram.
1. The annual modulation due to the Earth's orbital motion around the Sun.
1. The daily modulation due to the Earth's rotation can be seen if we zoom in. But, it's not clear sinusoidal pattern because the frequency resolution is not enough.

In [ ]:
tutorial_utils.plot_real_imag_spectrograms(
    timestamps["H1"], frequency, fourier_data["H1"]
)
fig, ax = tutorial_utils.plot_amplitude_phase_spectrograms(
    timestamps["H1"], frequency, fourier_data["H1"]
);

In [ ]:
# Zoom in the first 240 SFTs (240 x 1800 sec = 432000 sec = 5 days)
fig, ax = tutorial_utils.plot_amplitude_phase_spectrograms(
    timestamps["H1"], frequency, fourier_data["H1"],
);
ax[0].set_xlim([0, 240])
ax[1].set_xlim([0, 240])
ax[0].set_ylim([99.999, 100.005])
ax[1].set_ylim([99.999, 100.005])
fig.show()

## Create mock data to be analyzed by F-statistic (Sec.2) and Frequency Hough (Sec.3)

In [ ]:
depth = 5             # [Hz^{-1/2}]
sqrtSX = 1.0e-23      # [Hz^{-1/2}]
h0 = sqrtSX / depth

writer_kwargs = {
    "label": "SingleDetectorGaussianNoiseSignalInjected",
    "outdir": "Mockdata",
    "tstart": 1238166018,
    "duration": 0.4 * 365 * 86400,
    "detectors": "H1",
    "sqrtSX": sqrtSX,
    "Tsft": 1800,
    "SFTWindowType": "tukey",
    "SFTWindowParam": 0.01,
}

signal_parameters = {
    "F0": 100.0,    # GW frequency at initial time [Hz]
    "F1": -1e-9,    # first derivative of frequency (\dot{f}) [Hz/sec]
    "F2": 0.0,      # second derivative of frequency (\ddot{f}) [Hz/sec^2]
    "Alpha": 0.0,   # right ascention of the source [rad]
    "Delta": 0.0,   # declination angle of the source [rad]
    "h0": writer_kwargs["sqrtSX"] / depth,       # GW amplitude
    "cosi": 1,      # cosine of the inclination angle
    "psi": 0.0,     # polarization angle [rad]
    "phi": 0.0,     # initial phase [rad]
    "tref": writer_kwargs["tstart"],
}

writer = pyfstat.Writer(**writer_kwargs, **signal_parameters)
writer.make_data()
frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)
tutorial_utils.plot_real_imag_spectrograms(
    timestamps["H1"], frequency, fourier_data["H1"]
)
fig, ax = tutorial_utils.plot_amplitude_phase_spectrograms(
    timestamps["H1"], frequency, fourier_data["H1"]
);

In [ ]:
print(writer.sftfilepath)

# Include realistic effects (optional) <a name="include_realistic_effects"></a>

You can include non-stationary noise, gaps, line artifacts, and multiple detectors.

## Non-stationary noise<a name="non-stationarity"></a>

Real data, on the other hand, is hardly ever stationary over long periods of time.
This is equivalent to having a time-varying amplitude spectral density `sqrtSX`,
which can be easily implemented by running several instances of `pyfstat.Writer`
and concatenating the resulting file paths using `;`.

In [ ]:
segment_lengths = [5 * 86400, 3 * 86400, 4 * 86400]
segment_sqrtSX = [4e-23, 1e-23, 3e-23]

sft_path = []

# Setup Writer
writer_kwargs = {
    "outdir": "PyFstat_example_data",
    "tstart": 1238166018,
    "detectors": "H1",  # Detector to simulate, in this case LIGO Hanford
    "F0": 100.0,  # Central frequency of the band to be generated [Hz]
    "Band": 1.0,  # Frequency band-width around F0 [Hz]
    "sqrtSX": 1e-23,  # Single-sided Amplitude Spectral Density of the noise
    "Tsft": 1800,  # Fourier transform time duration
    "SFTWindowType": "tukey",
    "SFTWindowParam": 0.01,
}

for segment in range(len(segment_lengths)):
    writer_kwargs["label"] = f"Segment{segment}"
    writer_kwargs["duration"] = segment_lengths[segment]
    writer_kwargs["sqrtSX"] = segment_sqrtSX[segment]

    if segment > 0:
        writer_kwargs["tstart"] += writer_kwargs["Tsft"] + segment_lengths[segment - 1]

    writer = pyfstat.Writer(**writer_kwargs)
    writer.make_data()

    sft_path.append(writer.sftfilepath)

sft_path = ";".join(sft_path)  # Concatenate different files using ;
frequency, timestamps, fourier_data = get_sft_as_arrays(sft_path)

In [ ]:
tutorial_utils.plot_real_imag_spectrograms(
    timestamps["H1"], frequency, fourier_data["H1"]
);

## Gaps<a name="gaps"></a>

Moreover, interferometric detectors are not taking science-quality data throughout the full observing run,
meaning a real datastream will contain "gaps" during which no data is present.
These gaps can be either scheduled downtime periods to conduct maintenance in the detectors or the result of an
environmental perturbation driving the detector away from its operating point.

Functionality to simulate gaps is implemented via the `timestamps` keyword in `pyfstat.Writer`.
For each GPS timestamp, `pyfstat.Writer` will produce an SFT of data starting at such time and spanning
`Tsft` seconds. Mind that real data can contain arbitrarily long / short gaps, meaning the time span between
the end of an SFT and the beginning of the next one does *not* have to correspond to a multiple of `Tsft`.

In [ ]:
timestamps = {"H1": 1238166018 + 1800 * np.array([0, 2, 4, 6])}

# Setup Writer
writer_kwargs = {
    "label": "SingleDetectorGaps",
    "outdir": "PyFstat_example_data",
    "timestamps": timestamps,
    "F0": 100.0,  # Central frequency of the band to be generated [Hz]
    "Band": 1.0,  # Frequency band-width around F0 [Hz]
    "sqrtSX": 1e-23,  # Single-sided Amplitude Spectral Density of the noise
    "Tsft": 1800,  # Fourier transform time duration
    "SFTWindowType": "tukey",  # Window function to compute short Fourier transforms
    "SFTWindowParam": 0.01,  # Parameter associated to the window function
}
writer = pyfstat.Writer(**writer_kwargs)

# Create SFTs
writer.make_data()
frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

Special attention should be payed to the plotting function on use, as some functions like `pcolormesh`
may distort the plotting grid not to be squared.

In [ ]:
tutorial_utils.plot_real_imag_spectrograms_with_gaps(
    timestamps["H1"], frequency, fourier_data["H1"], writer_kwargs["Tsft"]
);

## Narrow instrumental artifacts<a name="lines"></a>

Another characteristic of real data that affects CW searches are
[persistent narrow instrumental artifacts](https://www.gw-openscience.org/O3/o3speclines/),
also known as *lines*,
which appear as strong, monochromatic, features in the detector data. Some of these lines have a known origin
(e.g. couplings to the power lines oscillating at 60 Hz,
vibrational modes of the mirror suspensions, LEDs blinking) but others are poorly understood.

This kind of artifacts can be simulated using `pyfstat.LineWriter`, which allows to specify the frequency
`F0`, initial phase `phi` and amplitude `h0` of the narrow instrumental artifact.

In [ ]:
writer_kwargs = {
    "label": "SingleDetectorSpectralLine",
    "outdir": "PyFstat_example_data",
    "tstart": 1238166018,  # Starting time of the observation [GPS time]
    "duration": 5 * 86400,  # Duration [seconds]
    "detectors": "H1",  # Detector to simulate, in this case LIGO Hanford
    "F0": 100.0,  # Central frequency of the band to be generated [Hz]
    "phi": 1.3,  # Initial phase of the spectral line
    "Band": 1.0,  # Frequency band-width around F0 [Hz]
    "h0": 1e-24,  # Amplitude of the spectral line
    "sqrtSX": 1e-23,  # Single-sided Amplitude Spectral Density of the noise
    "Tsft": 1800,  # Fourier transform time duration
    "SFTWindowType": "tukey",
    "SFTWindowParam": 0.01,
}

writer = pyfstat.LineWriter(**writer_kwargs)
writer.make_data()

frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

In [ ]:
tutorial_utils.plot_amplitude_phase_spectrograms(
    timestamps["H1"], frequency, fourier_data["H1"]
);

## Multiple detectors<a name="multiifo"></a>

Finally, the generation of data for multiple detectors is supported by specifying the detector names as a
comma-separated string in the `detectors` key word. The list of supported detectors includes, amongst others,
the advanced LIGO detectors (`H1` for LIGO Hanford, `L1` for LIGO Livingston)
and the advanced Virgo detector `V1`.

`sqrtSX` can be specified as a single, common value for all detectors or as a comma-separated string containing
one value for each of the specified detectors. Likewise, `timestamps` could be specified as a list of common
timestamps for all detectors or as a dictionary of lists, using detector names as keys.

In [ ]:
writer_kwargs = {
    "label": "SingleDetectorGaussianNoise",
    "outdir": "PyFstat_example_data",
    "tstart": 1238166018,  # Starting time of the observation [GPS time]
    "duration": 5 * 86400,  # Duration [seconds]
    "detectors": "H1,L1,V1",  # Detector to simulate, in this case LIGO Hanford
    "F0": 100.0,  # Central frequency of the band to be generated [Hz]
    "Band": 1.0,  # Frequency band-width around F0 [Hz]
    "sqrtSX": "1e-23,1e-24,1e-25",  # Single-sided Amplitude Spectral Density of the noise
    "Tsft": 1800,  # Fourier transform time duration
    "SFTWindowType": "tukey",  # Window function to compute short Fourier transforms
    "SFTWindowParam": 0.01,  # Parameter associated to the window function
}
writer = pyfstat.Writer(**writer_kwargs)

writer.make_data()

frequency, timestamps, fourier_data = get_sft_as_arrays(writer.sftfilepath)

In [ ]:
for ind, ifo in enumerate(timestamps.keys()):
    theoretical_stdev = np.sqrt(0.25 * writer_kwargs["Tsft"]) * float(
        writer_kwargs["sqrtSX"].split(",")[ind]
    )

    fig, ax = tutorial_utils.plot_real_imag_spectrograms(
        timestamps[ifo], frequency, fourier_data[ifo]
    )
    fig.suptitle(ifo)

    tutorial_utils.plot_real_imag_histogram(fourier_data[ifo], theoretical_stdev);